In [118]:
import re
total = 0
title = 0
length = 0
for i in open('NE_SciTfidf-30-20160118.txt','r'):
    if re.match('(CommentSample)(.)(.*)',i.strip()):
        title += 1
    else:
        length += 1
    total += 1
print "title",title,"length",length,"total",total
if (title+length) == total:
    print 'Read Over!\n'
else:
    print 'SomethingWrong!\n'

    
    
with open('NE_SciTfidf-30-20160118.txt','r') as fid:
    content = fid.read()
    from pprint import pprint
    Dict = {}
    c = 0
    for i in content.split('CommentSample\\'):
        if c > 0:
            d = 0
            Now = ''
            for o in i.split():
                if d == 0:
                    Now = o.strip()
                    Dict[Now] = []
                else:
                    Dict[Now].append(o.strip())                
                d+=1
        c+=1

DictFinal = {}
cnt = 0
domain = 'CommentSample\\'
for a in Dict:
    count = 0
    for u in open(domain+a,'r'):
        count+=1
        if count == 2:            
            DictFinal[u.strip()] = Dict[a]
            break
    #####印出原始字典#####
    #print a,'\n',str(Dict[a]).decode('string_escape')
    #####################
    for b in Dict[a]:
        cnt += 1

#####印出最終字典#####
for g in DictFinal:
    print g,'\n',str(DictFinal[g]).decode('string_escape')
#####################

print '\nWordsInDict',cnt
if cnt == length:
    print 'Finish! CheckOver!'
else:
    print 'Wrong!Need To Check!'

title 178 length 3848 total 4026
Read Over!

台灣高等法院 
['高等法院', '軍事博物館']
台北藝術大學 
['學位', '藝術', '區及', '平原']
捷運北門站 
['北門', '驛站', '壓迫', '高架橋', '懷舊']
蔡瑞月舞蹈研究社 
['乍眼', '研究社', '蔡瑞月', '鑄字', '日星', '夾著', '拐個', '木結構', '幽靜', '有趣']
永康街青田街 
['好吃', '思慕', '舒國治', '文藝', '度小月', '美食', '率性', '特別', '喜歡', '靈氣', '流連忘返', '清新', '適宜', '著名', '札記', '愜意', '七六', '熱鬧', '母上', '安靜', '臺大', '飯館', '街巷', '有意思', '大來', '麗水', '給我二', '甘味', '小月', '昔冰']
梅庭 
['梅庭', '于右任', '右任', '寓所', '草聖', '綠樹成蔭', '梅亭', '教育家', '復旦大學', '質朴', '別館', '書法作品', '雅緻', '舊居', '幽雅', '溪流', '泡溫泉', '住處', '北投區', '風景', '即到', '優雅', '紀念館', '舒適', '恭賀', '開敞', '蒼鬱', '肖像畫', '領些', '指指點點']
中研院歷史文物陳列館 
['殷墟', '陳列館', '方鼎', '出土', '胡適', '大墓', '大庫', '全型', '碑拓', '第一遍', '星散', '王陵', '居延', '百不存一', '盜掘', '漢簡', '紀念品', '講到', '碑帖', '胡適紀念館', '毛公鼎', '點看', '僅存', '珍貴文物', '偏遠', '研究所', '歷史文物', '雕像', '精美', '免費參觀']
告白巷 
['告白', '主唱', '五迷', '喜歡', '初戀', '五月天', '沒有勇氣', '走紅', '巷有', '巷耶', '北極熊', '冰有', '谷好', '一堵', '喊出', '有趣', '花樹', '駛過', '七拐八拐', '普普通通', '表白', '偏好', '巡禮', '傻傻的', '好評', '簽到', '妹紙', '白牆

In [128]:
import gensim
#1__WIKI
model = gensim.models.Word2Vec.load("W2V_Wiki_Trastd/wiki.zh.sim.seg.trastd.model")
def W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent):
    import operator
    cnt = []#　　計算餵進來的總詞量
    Expt = []#　　存W2V無法辨識的詞語
    Remain = []#　　存沒有通過門檻的詞語
    ClassingResult = {}

    for z in Concept:#　　把概念標頭寫進字典
        ClassingResult[z] = {}#　　每個字典的內容都是一個字典

    for q in open(Corpus,'r'):#　　把萃取關鍵字的結果讀出來        
        q = q.strip().decode('utf-8')#　　去空白,解碼
        cnt.append(q)#　　把讀取內容寫進一陣列(計算樣本的資料量用)
        try:
            TmpDic = {}#　　暫時性的字典(比對詞語與各概念的相似度)
            SameConcept = ''#　　用來存放概念,當q直接是某概念時使用
            TmpWord = 'None'#　　用來比對詞語是不是與概念完全吻合,若是,None會被修改掉;反之則維持None(初值)

            #print '@ ->「',q,'」的概念分類開始: '

            for a in Concept:#　　讀取各概念標頭
                TmpList = []#　　暫時性的陣列(存放'詞語'vs'概念內各詞語'的相似度)     
                for b in Concept[a]:#　　讀取個概念內的詞語

                    #####完全ㄧ樣#####
                    if q.encode('utf-8') == b:#　　如果等於
                        #print '---目標詞「',q,'」就在【',a,'】的概念裡'
                        TmpWord = b#　　把詞語指定給TmpWord
                        break#　　跳出迴圈(已經知道跟某詞語完全一樣,就不用再比其他的詞語了)
                    #####

                    if model.similarity(q,b.decode('utf-8')) > SimLevel:#　　如果相似度達門檻
                        TmpList.append(model.similarity(q,b.decode('utf-8')))#　　就把相似度寫進TmpList裡面

                #####完全一樣#####        
                if TmpWord != 'None':#　　不是None(初值)->被修改過->要完全相等才會出現此情況
                    SameConcept = a#　　把現在比對到的概念( EX:輕鬆 )記起來
                    break#　　跳出迴圈(已經知道是屬於哪個概念,就不用再比對其他概念了)
                #####

                #####印出q與一概念比較的詳細數據
                #print '  -目標詞「',q,'」與【',a,'】的相似陣列( 顯示內容為已高於相似度門檻 :',SimLevel,')'
                #print TmpList
                #print '  -其占【',a,'】概念內容的\n   比重為 :',len(TmpList),'/',len(Dic[a])
                #print '   比率為 :',
                #print float(len(TmpList))/float(len(Concept[a]))
                #print '*'
                #####

                if float(len(TmpList))/float(len(Concept[a])) > SimPercent:#　　如果達到相似成數的門檻
                    #print '  -「',q,'」已過相似_百分比_門檻 :',SimPercent
                    TmpAvg = 0.0#　　存放平均相似度
                    for c in TmpList:#　　把每個相似度讀出來
                        TmpAvg += c#　　加起來
                    TmpDic[a] = float(TmpAvg)/float(len(TmpList))#　　把'總數'除以'陣列長度',得到平均相似度,
                                                                    #再寫進字典裡,當q與多個概念都有相似度食
                                                                    #可以比較與哪個概念的相似度比較高
                    #print '   與【',a,'】的相似度為: ',TmpDic[a],' -> 進入候選名單'
                    #print '# # # # # # #\n'

            #####完全一樣##### 
            if TmpWord != 'None': 
                ClassingResult[SameConcept][(q.encode('utf-8'))] = 1#　　把q寫進總字典的概念裡(Key:q Value:1)
                continue
            #####

            #####檢視暫存字典#####
            #print '\n___檢視',q,'的暫存字典(TmpDic)___'
            #for k in TmpDic:#　　遍歷暫存字典,看q與個概念的相似度
            #    print k,TmpDic[k]
            #print '== == == == == == == == == == =='
            #####

            if len(TmpDic) > 0:#　　如果暫存字典有任何一概念(表示q'至少'與一個概念相似)
                #print '> >小結 :',q,'成功歸類！',
                for d in TmpDic:#　　遍歷TmpDic
                    if d == sorted(TmpDic.iteritems(),key=operator.itemgetter(1),reverse=True)[0][0]:
                        #如果d = 排序後的第一名 (表示d就是與q最像的概念)
                        if TmpDic[d] > SimLevel:#　　如果"q與概念的相似度"大於相似度門檻
                            #print '類別為 :',d,'\n'
                            ClassingResult[d][q.encode('utf-8')] = TmpDic[d]#　　把q寫進總字典的概念裡(Key:q Value:相似度)
                        else:#　　不太可能發生的情況
                            Remain.append('WHAT!?'+q.encode('utf-8'))                    
            else:#　　q與各概念相似度都沒有達標,無法歸類
                Remain.append(q.encode('utf-8'))

        except KeyError:#　　抓到KeyError( W2V不認識q )的例外處理
            #print '「',q,'」EXCEPT! (KeyError)\n'
            Expt.append(q.encode('utf-8'))#　　把q加進例外的陣列理
    return ClassingResult

In [130]:
Dic = {
'藝文':['藝術','美術','戲劇','音樂'],\
    
'信仰':['信仰','祭祀','祭拜','膜拜','信眾','信奉'],\
    
'美食':['美食','爽口','美味','好吃'],\
'新奇':['新奇','新穎','獵奇','獨特','驚喜'],\

'輕鬆':['輕鬆','休閒','遊憩','悠閒','休憩','消遣','愜意','放鬆'],\
'遊玩':['遊玩','好玩','遊樂'],\
'快樂':['快樂','愉快','開心','歡樂','幸福','美好'],\
'有趣':['有趣','趣味'],\

'脫俗':['脫俗','幽雅','清幽','優雅','氣質','心曠神怡'],\
'樂活':['環保','綠化','清新'],\

'安靜':['安靜','寧靜'],\
'乾淨':['乾淨','整潔','潔淨'],\

'懷舊':['懷舊','復古','情調'],\
'古老':['古老','悠久'],\

'髒亂':['骯髒','很髒','髒亂','低劣','灰塵','破舊'],\

'購物':['商圈','百貨','商場'],\
'特產':['特產','紀念品'],\

'運動':['運動','健康'],\

'漂亮':['漂亮','可愛','美麗','優美'],\
'文青':['清新','優雅','氣質'],\

'壯觀':['雄偉','壯觀','壯麗']
}

In [131]:
ClassingResult = W2V_ClassificationMethod2(Dic,'NE_SciTfidf-30-20160118.txt',0.35,0.49)


遊玩 {'遊玩': 1, '好玩': 1, '遊樂': 1}
新奇 {'想像力': 0.43064241505868656, '豐富': 0.4509356715801665, '驚喜': 1, '表現力': 0.42818609634164245, '獨特': 1, '現代感': 0.49488485715569563}
懷舊 {'復古': 1, '情感世界': 0.4535131719488257, '悠揚': 0.4416992669416935, '別具一格': 0.4032206920639193, '懷舊': 1}
乾淨 {'舒適': 0.39928212581537637, '整潔': 1, '乾淨': 1, '水質': 0.3720850214532764}
藝文 {'戲劇': 1, '藝術': 1, '音樂': 1, '戲曲': 0.49011586151432945, '偶戲': 0.3675229526902668, '攝影': 0.5631884510979743, '雕塑': 0.47193195897590756, '花藝': 0.35236732581901803, '創意': 0.44984862638870626, '美術': 1}
信仰 {'虔誠': 0.5009075344501047, '湯守': 0.35690754287699994, '信仰': 1, '善男信女': 0.37082087632450805, '城隍': 0.47197734037115086, '宗教': 0.527963450370827, '神靈': 0.49279764271267884, '寺廟': 0.40795445270931524, '祖師': 0.43999985288167637, '金紙': 0.4065994632160217, '廟宇': 0.5482696953339619, '靈驗': 0.5032411999770633, '媽祖廟': 0.39193661420182846, '祈福': 0.49170126301354206, '神明': 0.5645932082687286, '跪拜': 0.388789287460657, '香客': 0.5321886414380881, '祭儀': 0.453899270290

In [132]:
for ee in ClassingResult:
    print ee,str(ClassingResult[ee]).decode('string_escape')

遊玩 {'遊玩': 1, '好玩': 1, '遊樂': 1}
新奇 {'想像力': 0.43064241505868656, '豐富': 0.4509356715801665, '驚喜': 1, '表現力': 0.42818609634164245, '獨特': 1, '現代感': 0.49488485715569563}
懷舊 {'復古': 1, '情感世界': 0.4535131719488257, '悠揚': 0.4416992669416935, '別具一格': 0.4032206920639193, '懷舊': 1}
乾淨 {'舒適': 0.39928212581537637, '整潔': 1, '乾淨': 1, '水質': 0.3720850214532764}
藝文 {'戲劇': 1, '藝術': 1, '音樂': 1, '戲曲': 0.49011586151432945, '偶戲': 0.3675229526902668, '攝影': 0.5631884510979743, '雕塑': 0.47193195897590756, '花藝': 0.35236732581901803, '創意': 0.44984862638870626, '美術': 1}
信仰 {'虔誠': 0.5009075344501047, '湯守': 0.35690754287699994, '信仰': 1, '善男信女': 0.37082087632450805, '城隍': 0.47197734037115086, '宗教': 0.527963450370827, '神靈': 0.49279764271267884, '寺廟': 0.40795445270931524, '祖師': 0.43999985288167637, '金紙': 0.4065994632160217, '廟宇': 0.5482696953339619, '靈驗': 0.5032411999770633, '媽祖廟': 0.39193661420182846, '祈福': 0.49170126301354206, '神明': 0.5645932082687286, '跪拜': 0.388789287460657, '香客': 0.5321886414380881, '祭儀': 0.453899270290

In [144]:
DictResult={}
for a in DictFinal:
    DictResult[a] = {'概　念':{},'文字雲':[]} 
    for b in DictFinal[a]:
        z = 'Homeless'
        for c in ClassingResult:
            if b in ClassingResult[c]:
                z = 'Home'
                if not c in DictResult[a]['概　念']:
                    DictResult[a]['概　念'][c] = 1
                else:
                    DictResult[a]['概　念'][c] += 1
                break
        if z == 'Homeless':
            DictResult[a]['文字雲'].append(b)

In [190]:
#印出完整結果
for g in DictResult:
    print '【',g,'】'
    for h in DictResult[g]:
        print h,str(DictResult[g][h]).decode('string_escape')
    print ''

【 新光摩天大樓 】
概　念 {}
文字雲 ['區位', '摩天大樓', '眺望臺', '特別']

【 松山慈祐宮 】
概　念 {'古老': 1, '購物': 1, '信仰': 5, '運動': 2, '壯觀': 1, '新奇': 1, '美食': 1, '有趣': 1}
文字雲 ['急難救助', '仁懷', '冬令救濟', '人溺己溺', '橫下心來', '獎助學金', '績優', '雕刻', '第三級', '自建', '慈悲', '廟口', '值得一試', '常年', '古蹟', '專修', '特維']

【 台北藝術大學 】
概　念 {'藝文': 1}
文字雲 ['學位', '區及', '平原']

【 捷運北門站 】
概　念 {'懷舊': 1}
文字雲 ['北門', '驛站', '壓迫', '高架橋']

【 蔡瑞月舞蹈研究社 】
概　念 {'有趣': 1, '脫俗': 1}
文字雲 ['乍眼', '研究社', '蔡瑞月', '鑄字', '日星', '夾著', '拐個', '木結構']

【 永康街青田街 】
概　念 {'漂亮': 1, '古老': 1, '樂活': 1, '脫俗': 1, '安靜': 1, '美食': 2, '有趣': 2, '文青': 1, '輕鬆': 1}
文字雲 ['思慕', '舒國治', '文藝', '度小月', '特別', '靈氣', '適宜', '札記', '七六', '母上', '臺大', '飯館', '街巷', '大來', '麗水', '給我二', '甘味', '小月', '昔冰']

【 梅庭 】
概　念 {'壯觀': 1, '脫俗': 2, '乾淨': 1}
文字雲 ['梅庭', '于右任', '右任', '寓所', '草聖', '梅亭', '教育家', '復旦大學', '質朴', '別館', '書法作品', '雅緻', '舊居', '溪流', '泡溫泉', '住處', '北投區', '風景', '即到', '紀念館', '恭賀', '開敞', '蒼鬱', '肖像畫', '領些', '指指點點']

【 中研院歷史文物陳列館 】
概　念 {'特產': 2, '壯觀': 1}
文字雲 ['殷墟', '陳列館', '方鼎', '出土', '胡適', '大墓', '大庫', '全型', '碑拓', '第一遍', '星散', 

In [188]:
#只印出所屬概念
for g in DictResult:
    print '【',g,'】',
    for u in DictResult[g]['概　念']:
        print u,
    print '\n'


【 新光摩天大樓 】 

【 松山慈祐宮 】 古老 購物 信仰 運動 壯觀 新奇 美食 有趣 

【 台北藝術大學 】 藝文 

【 捷運北門站 】 懷舊 

【 蔡瑞月舞蹈研究社 】 有趣 脫俗 

【 永康街青田街 】 漂亮 古老 樂活 脫俗 安靜 美食 有趣 文青 輕鬆 

【 梅庭 】 壯觀 脫俗 乾淨 

【 中研院歷史文物陳列館 】 特產 壯觀 

【 告白巷 】 髒亂 漂亮 壯觀 有趣 安靜 

【 西門町商圈 】 漂亮 古老 購物 遊玩 美食 有趣 藝文 

【 台北故事館 】 漂亮 文青 懷舊 

【 台北市孔廟 】 有趣 信仰 懷舊 

【 台北霞海城隍廟 】 特產 信仰 美食 

【 台灣高等法院 】 

【 四平公園 】 有趣 購物 安靜 美食 漂亮 

【 松山彩虹橋 】 漂亮 購物 藝文 

【 閻錫山墓 】 

【 台北琴道館 】 懷舊 

【 清水巖祖師廟 】 古老 信仰 髒亂 安靜 有趣 文青 

【 滾石國際音樂有限公司 】 安靜 藝文 

【 台北龍山寺 】 文青 壯觀 信仰 購物 有趣 

【 國父紀念館 】 特產 有趣 文青 壯觀 脫俗 

【 台北車站 】 美食 有趣 購物 壯觀 乾淨 

【 象山 】 漂亮 運動 安靜 髒亂 壯觀 有趣 藝文 

【 延平河濱公園 】 漂亮 安靜 

【 外交部 】 壯觀 安靜 文青 遊玩 

【 台北關渡宮 】 漂亮 古老 信仰 壯觀 美食 有趣 

【 台北101 】 漂亮 古老 購物 安靜 美食 特產 有趣 

【 富陽自然生態公園 】 樂活 信仰 乾淨 

【 世新大學 】 特產 安靜 美食 

【 北投文物館 】 安靜 文青 脫俗 

【 芝山文化生態綠園 】 購物 

【 撫台街洋樓 】 

【 饒河夜市足沐腳底按摩 】 文青 購物 輕鬆 

【 忠烈祠 】 有趣 信仰 脫俗 

【 寶藏巖國際藝術村 】 漂亮 有趣 藝文 

【 公館水岸公園 】 購物 

【 北投公園 】 古老 乾淨 樂活 安靜 有趣 文青 輕鬆 

【 鳳甲美術館 】 快樂 安靜 脫俗 壯觀 特產 有趣 

【 美麗華百樂園 】 漂亮 快樂 購物 有趣 脫俗 

【 慈諴宮 】 文青 信仰 購物 

【 台北市議會圖書館 】 有趣 脫俗 

【 天后宮 】 文青 壯觀 信仰 

【 自來水園區 】 有趣

In [167]:
import operator
w = sorted(DictResult.iteritems(),key=operator.itemgetter(1),reverse=True)
for i in DictResult.iteritems():
    print i
    print type(operator.itemgetter(1))

SyntaxError: invalid syntax (<ipython-input-167-bf887ac639d6>, line 3)

In [187]:
dic = {'o':{'概　念':{1:1,2:2,3:3},5:1},\
       'p':{'概　念':{2:2},7:1},\
       'w':{'概　念':{3:3},3:1}}
for i in sorted(dic.iteritems(),key=operator.itemgetter(1,len(1)),reverse=True):
    print i

TypeError: object of type 'int' has no len()

In [127]:
d = {'概念':{'脫俗':0.115},'文字雲':[]}
if '脫俗' in d['概念']:
    print 'ddd'

ddd
